# Story-writing with BLOOM

SPDX-License-Identifier: 0BSD

In [1]:
from collections.abc import Callable
import functools
import random
import re
import textwrap
from typing import Any

from huggingface_hub import InferenceApi

In [2]:
_PARA_BREAK = re.compile(r'\n{2,}')
"""Regular expression matching a paragraph break."""

'Regular expression matching a paragraph break.'

In [3]:
class CompletionError(Exception):
    """An error in attempting to complete text with BLOOM."""
    pass

In [4]:
class UnexpectedResponseError(AssertionError):
    """An unexpected error completing text. Probably a bug."""
    pass

In [5]:
class Supplier:
    """As the value of a BLOOM parameter, is called instead of used."""

    __slots__ = ('_func',)

    __match_args__ = ('_func',)

    def __init__(self, func: Callable[[], Any]) -> None:
        self._func = func
    
    def __repr__(self) -> str:
        return f'{type(self).__name__}({self._func!r})'
    
    def __call__(self) -> Any:
        return self._func()

In [6]:
@functools.cache
def _get_inference_api() -> InferenceApi:
    with open('.hf_token', encoding='utf-8') as file:
        api_token = file.read().strip()

    return InferenceApi(repo_id='bigscience/bloom', token=api_token)

In [7]:
def _normalize_paragraphs(text: str) -> str:
    """Remove any hard wrapping and separate paragraphs by single newlines."""
    pretty = _PARA_BREAK.split(text)
    raw = (para.strip().replace('\n', ' ') for para in pretty)
    return '\n'.join(raw)

In [8]:
def _prettify_paragraphs(text: str) -> str:
    """Add hard wrapping and separate paragraphs by blank lines."""
    raw = text.strip().split('\n')
    pretty = ('\n'.join(textwrap.wrap(graf)) for graf in raw)
    return '\n\n'.join(pretty)

In [9]:
class Completer:
    """High-level interface to Bloom completion."""

    __slots__ = ('_inference', '_text', '__dict__')

    def __init__(self, prompt: str, **parameters: Any) -> None:
        """Create a completer for the given prompt and optional parameters."""
        self._inference = _get_inference_api()
        self.text = _normalize_paragraphs(prompt)

        if any(name.startswith('_') for name in parameters):
            raise TypeError('cannot set parameter name with leading "_"')
        
        self._set_defaults()
        self.__dict__.update(parameters)

    def __str__(self) -> str:
        """Prettified version of the text stored so far."""
        return _prettify_paragraphs(self.text)
    
    @property
    def text(self) -> str:
        """The prompt plus any appended completions."""
        return self._text
    
    @text.setter
    def text(self, value: str) -> None:
        if not isinstance(value, str):
            raise TypeError('prompt must be a string')
        if not value:
            raise ValueError('prompt must be nonempty')
        self._text = value

    def __call__(self) -> None:
        """Attempt to complete the stored text, and print all text so far."""
        self.complete()
        print(self)
    
    def complete(self) -> None:
        """Attempt to complete the stored text, but do not print anything."""
        match self._inference(inputs=self.text, params=self._build_params()):
            case [{'generated_text': completion}]:
                self._text = completion  # Whatever BLOOM returns, we accept.
            case {'error': [*errors]}:
                raise CompletionError(*errors)
            case other_response:
                raise UnexpectedResponseError(other_response)
    
    def _set_defaults(self) -> None:
        self.do_sample = True
        self.max_new_tokens = 250
        self.seed = Supplier(random.random)
        self.temperature = 0.75
    
    def _build_params(self) -> dict[str, Any]:
        return {name: (value() if isinstance(value, Supplier) else value)
                for name, value in sorted(self.__dict__.items())}

In [10]:
bloom1 = Completer("""
Sometimes I marvel at how the sky contains no advertisements. Oh, sometimes
someone goes up in a little plane and skywrites a birthday greeting. But the
sky... you don't have to watch an ad to see it.

When people learn I fought in the war to free the sky, they say, "Thank you for
""", temperature=1.0, top_k=30)

In [11]:
bloom1()

Sometimes I marvel at how the sky contains no advertisements. Oh,
sometimes someone goes up in a little plane and skywrites a birthday
greeting. But the sky... you don't have to watch an ad to see it.

When people learn I fought in the war to free the sky, they say,
"Thank you for not having ads." I'm sorry for that, but I don't really
see it that way. I thought the sky was just a big, good field,
something for nothing. And that's what I fought for, not to keep it
free of commercial messages.

This is not true. My company has a television advertising budget. We
have a website that's full of advertisements. I don't even watch any
TV. But the fact is that advertisers pay us for the right to display
their ads. And we don't get it for nothing - we have to run an ad.

Our ad server collects a lot of data about the ads that run, and the
devices that run them, and how many people they reach. One thing it's
amazing at doing is giving you a good idea of how many ads people see.
(To be precise, 

In [12]:
bloom2 = Completer("""
The color enforcers came around again. They ripped the cheese off our freshly
baked pizza. The customers dropped their food and ran out.

"Aha!" said the biggest, scariest henchman. "This sauce is red! We license the
color red to you, but you haven't been paying the license fees." I quaked in
fear. My grandfather had gotten both his knees busted by color enforcers.
""")

In [13]:
bloom2()

The color enforcers came around again. They ripped the cheese off our
freshly baked pizza. The customers dropped their food and ran out.

"Aha!" said the biggest, scariest henchman. "This sauce is red! We
license the color red to you, but you haven't been paying the license
fees." I quaked in fear. My grandfather had gotten both his knees
busted by color enforcers. Now they were coming for me.

Just then the pizza chef came out. He had a big, red mustache and a
red shirt. He was a real badass. He kicked the color enforcer right in
the groin. I felt so proud that I was his granddaughter.

The enforcer fell to the ground, groaning. We ran out of the other
door. The owner of the restaurant was waiting for us. I felt like I
was in a movie.


In [14]:
bloom3 = Completer("""
Beverly understood how all the women in her family had fallen in love with this
energy ghost who lived in a lamp and fed parasitically off their vital energy
while making every carnal dream come to life. As a spacefaring physician,
""")

In [15]:
bloom3()

Beverly understood how all the women in her family had fallen in love
with this energy ghost who lived in a lamp and fed parasitically off
their vital energy while making every carnal dream come to life. As a
spacefaring physician, she knew that the phi’s energy was derived from
the same organ that had driven her people to the stars. It was the
organ that allowed her people to generate the brain power required to
launch interstellar travel and lived in the center of their brains
where it could upgrade their minds to the level of a computer’s
processing power. However, it was also the organ that caused the organ
death of all phi doppelgangers once their hosts died.

Luckily, if a phi was left alone for a thousand years, they could
fully mature and become a full fledged entity on their own. On the
other hand, if they were disturbed, they would return to child-like
status and only grow again if left alone for another thousand years.
There was no way to draw them out of their child-like st

In [16]:
# From https://aclanthology.org/2020.acl-main.463.pdf, Appendix A.
bloom4 = Completer("""
Robinson cried out, “Help! I’m being chased by a bear! All I have is these
sticks. What should I do?” The survivalist said, “Here’s what you can do.
""")

In [17]:
bloom4()

Robinson cried out, “Help! I’m being chased by a bear! All I have is
these sticks. What should I do?” The survivalist said, “Here’s what
you can do. Take one of these sticks and, as you run, throw it as far
as you can in front of you. The bear will then chase that stick
thinking you’re still right behind it. Take the other stick and run
with it back in the direction you came from. The bear will think you
ran that way and will then chase it back to your original location.”
Robinson took the advice and was able to escape the bear. He said,
“I’m so thankful that I learned this trick from you.”
